# 图像卷积

互相关运算

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K): # X: 输入的二维张量（如图像矩阵）；K: 卷积核（kernel/filter），也是二维张量  
    """计算二维互相关运算"""
    # 1. 获取卷积核的尺寸（高度和宽度）
    h, w = K.shape
    # 2. 创建输出张量，尺寸自动计算
    # (输入高 - 卷积核高 + 1, 输入宽 - 卷积核宽 + 1)
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    # 3. 遍历输出矩阵的每个位置
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            '''
            4. 计算当前位置的互相关值
            X[i:i+h, j:j+w]: 从输入中提取与卷积核大小相同的子区域（感受野）
            * K: 将子区域与卷积核逐元素相乘
            .sum(): 将所有乘积相加，得到当前位置的响应值
            '''
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    # 5. 返回结果
    return Y

验证上述二维互相关运算的输出，输出大小等于：$(n_{k}-k_{h}+1)\times (n_{w}-k_{w}+1)$，<br>0 x 0+1 x 1+3 x 2+4 x 3=19，<br>1 x 0+2 x 1+4 x 2+5 x 3=25，<br>3 x 0+4 x 1+6 x 2+7 x 3=37，<br>4 x 0+5 x 1+7 x 2+8 x 3=43，<br>![本地路径](./img/图像卷积.png)

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

实现二维卷积层

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__() # 调用父类nn.Module的构造函数
        # 1. 初始化可学习的权重参数（卷积核）
        # torch.rand(kernel_size) 生成指定形状的随机值（范围[0,1)）
        # nn.Parameter 将其标记为模型参数，训练时会自动更新
        # kernel_size可以是元组如 (2,3) 或单个整数如3（表示3×3的核）
        self.weight = nn.Parameter(torch.rand(kernel_size))
        # 2. 初始化可学习的偏置参数
        # 初始化为0，形状为(1,)的标量
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        # 3. 定义前向传播过程
        # corr2d(x, self.weight) 计算互相关（即卷积操作）
        # + self.bias 给每个输出元素加上偏置值
        return corr2d(x, self.weight) + self.bias

卷积层的一个简单应用：
检测图像中不同颜色的边缘

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
# [[1.0, -1.0]]是一个嵌套列表，表示一个1行2列的二维数组
K = torch.tensor([[1.0, -1.0]])

输出`Y`中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘

In [6]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核`K`只可以检测垂直边缘

In [7]:
# X.t()：对 X 进行转置
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习由`X`生成`Y`的卷积核

In [8]:
'''
1. 创建一个1x2的卷积层（无偏置）
1: 输入通道数（灰度图像）
1: 输出通道数
kernel_size=(1,2): 1行2列的卷积核
bias=False: 不使用偏置项
'''
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)
# 2. 调整输入X的形状为(批量大小, 通道数, 高度, 宽度)
X = X.reshape((1, 1, 6, 8))
# 3. 调整目标输出Y的形状
Y = Y.reshape((1, 1, 6, 7))
# 4. 设置学习率
lr = 3e-2
# 5. 训练循环（10轮）
for i in range(10):
    # 6. 前向传播：计算预测值
    Y_hat = conv2d(X)
    # 7. 计算损失（均方误差）
    l = (Y_hat - Y) ** 2
    # 8. 清空梯度
    conv2d.zero_grad()
    # 9. 反向传播：计算梯度
    l.sum().backward()
    # 10. 手动更新权重（梯度下降）
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    # 11. 每2轮打印损失
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 11.308
epoch 4, loss 2.017
epoch 6, loss 0.387
epoch 8, loss 0.085
epoch 10, loss 0.022


所学的卷积核的权重张量

In [9]:
'''
conv2d.weight：访问Conv2d层的卷积核权重：这是一个nn.Parameter对象（可学习的参数）
.data：获取参数的底层张量数据
with torch.no_grad():
    conv2d.weight.reshape((1, 2))
'''
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9951, -0.9707]])